In [2]:
import pandas as pd

# Load 'twcs' from 'raw' file
df = pd.read_csv(r'C:\Users\User\projects\ai_support_assistant\data\raw\twcs.csv')

In [3]:
# filter for 'inbund' == True tweets
df = df[df['inbound']==False]

In [5]:
company_names = df['author_id'].unique()

In [12]:
import pandas as pd

# unique company screen names from 'author_id' where inbound == False
outbound_df = df[df['inbound'] == False]
company_names = outbound_df['author_id'].unique()

In [ ]:
import re
from collections import Counter

# @-mentions from 'text' where inbound == True
inbound_df = df[df['inbound'] == True]
mentions = []
for text in inbound_df['text']:
    mentions.extend(re.findall(r'@\w+', text))  # Extract @mentions

# Remove @ prefix from mentions
mentions = [mention.lstrip('@') for mention in mentions]

# Count the frequency of mentions that are in the company_names list
mention_counts = Counter([mention for mention in mentions if mention in company_names])

# Get the top 10 mentioned company names
target_companies_list = [mention for mention, _ in mention_counts.most_common(10)]

print("\nTarget Companies List:")
print(target_companies_list)


Target Companies List:
['AmazonHelp', 'AppleSupport', 'AmericanAir', 'Uber_Support', 'Delta', 'VirginTrains', 'SouthwestAir', 'Tesco', 'SpotifyCares', 'British_Airways']


In [ ]:
# Function to extract the first valid @-mention from the text that is in target_companies_list
def extract_company_label(text, target_companies):
    mentions = re.findall(r'@\w+', text)  # Extract all @mentions
    for mention in mentions:
        mention_cleaned = mention.lstrip('@')  # Remove @ prefix
        if mention_cleaned in target_companies:
            return mention_cleaned
    return None

# Create a new column for cleaned text by removing the extracted company label
def clean_text(text, label):
    if label:
        return re.sub(rf'@{label}', '', text).strip()  # Remove the @mention from the text
    return text

In [ ]:
# Apply the functions to create a new columns
inbound_df['extracted_company_label'] = inbound_df['text'].apply(lambda x: extract_company_label(x, target_companies_list)) 
inbound_df['cleaned_text_for_feature'] = inbound_df.apply(
    lambda row: clean_text(row['text'], row['extracted_company_label']), axis=1

In [ ]:
# Filter out rows where extracted_company_label is null
filtered_df = inbound_df[inbound_df['extracted_company_label'].notnull()]

In [ ]:
# download nltk for text cleaning
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# Text cleaning function
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a single string
    return ' '.join(tokens)

In [ ]:
# Apply text cleaning to the 'cleaned_text_for_feature' column
filtered_df['cleaned_text_for_feature'] = filtered_df['cleaned_text_for_feature'].apply(clean_text_for_model)

In [ ]:
# Encode the 'extracted_company_label' column using LabelEncoder
label_encoder = LabelEncoder()
filtered_df['encoded_company_label'] = label_encoder.fit_transform(filtered_df['extracted_company_label'])